In [1]:
import pandas as pd
import numpy as np
import h5py
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, Dropout, Reshape

2024-11-02 09:07:15.928272: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
file = h5py.File('xy.h5', 'r')

x = file['x'][:]
y = file['y'][:]

x.shape, y.shape

((50000, 3675), (50000,))

In [3]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((40000, 3675), (40000,), (10000, 3675), (10000,))

In [ ]:
input = Input(shape=(3675,))
embedding = Embedding(input_dim=4096, output_dim=1)(input)
flatten = Flatten()(embedding)
dropout = Dropout(.25)(flatten)
output = Dense(1, activation='sigmoid')(dropout)

model = Model(inputs=input, outputs=output)

In [5]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3675)]            0         
                                                                 
 embedding (Embedding)       (None, 3675, 1)           4096      
                                                                 
 flatten (Flatten)           (None, 3675)              0         
                                                                 
 dropout (Dropout)           (None, 3675)              0         
                                                                 
 dense (Dense)               (None, 1)                 3676      
                                                                 
Total params: 7772 (30.36 KB)
Trainable params: 7772 (30.36 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
optimizer = Adam(learning_rate=.0001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy','mse'])
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=32, batch_size=4096)

Epoch 1/32
10/10 [==============================] - 6s 516ms/step - loss: 0.6942 - accuracy: 0.4951 - mse: 0.2505 - val_loss: 0.6931 - val_accuracy: 0.4994 - val_mse: 0.2500
Epoch 2/32
10/10 [==============================] - 4s 417ms/step - loss: 0.6936 - accuracy: 0.5062 - mse: 0.2502 - val_loss: 0.6931 - val_accuracy: 0.5003 - val_mse: 0.2500
Epoch 3/32
10/10 [==============================] - 5s 443ms/step - loss: 0.6937 - accuracy: 0.5032 - mse: 0.2503 - val_loss: 0.6931 - val_accuracy: 0.5025 - val_mse: 0.2500
Epoch 4/32
10/10 [==============================] - 4s 430ms/step - loss: 0.6933 - accuracy: 0.5045 - mse: 0.2501 - val_loss: 0.6931 - val_accuracy: 0.5045 - val_mse: 0.2500
Epoch 5/32
10/10 [==============================] - 4s 440ms/step - loss: 0.6935 - accuracy: 0.4996 - mse: 0.2502 - val_loss: 0.6931 - val_accuracy: 0.5082 - val_mse: 0.2500
Epoch 6/32
10/10 [==============================] - 4s 429ms/step - loss: 0.6936 - accuracy: 0.5016 - mse: 0.2502 - val_loss: 0.69

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

plt.show()